In [3]:
import pyspark.sql.functions as f

StatementMeta(, 5a1b6635-514b-4349-a745-6e7252984f75, 5, Finished, Available, Finished)

### Change activity ids to upper case
In the new version specific IDs in the activity table have been put to Upper Case to be able to join to other tables in Power BI. This script updates these IDs one time

In [5]:
df_activities = spark.sql("SELECT * FROM FUAM_Lakehouse.activities")
for co in df.columns:
    if co in ['ActivityId','ArtifactId','CapacityId','DashboardId','DataflowId','DatasetId','DatasourceId','FolderObjectId','GatewayId','Id','ItemId','ReportId','UserId','WorkspaceId','','','','','','','','','',]:
        df_activities = df_activities.withColumn(co, f.upper(df_activities[co]))
df_activities.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("activities")

StatementMeta(, 5a1b6635-514b-4349-a745-6e7252984f75, 7, Finished, Available, Finished)

### Delete silver tables
Old silver tables will be deleted, since they are now available in a seperate Lakehouse

In [7]:
tables_to_delete = ['capacities_silver','active_items_silver','tenant_settings_silver','delegated_tenant_settings_overrides_silver','workspaces_silver','capacity_metrics_by_item_kind_by_day_silver','capacity_metrics_by_timepoint_silver','capacity_metrics_by_item_by_operation_by_day_silver']

for table in spark.catalog.listTables():
    table_name = table[0]
    if table_name in tables_to_delete:
        print(table_name + ' will be deleted')
        spark.sql("DROP TABLE "+ table_name)

StatementMeta(, 5a1b6635-514b-4349-a745-6e7252984f75, 9, Finished, Available, Finished)

### Move old files into new file structure
Existing folders in the lakehouse will be moved to the history folder to enable better readability. The will be just put into an "old" folder and not completely be migrated to the new folder structure

In [2]:
folders_to_move = ['active_items','activities','capacities','delegated_tenant_settings_overrides','inventory','refreshables','tenant_settings','workspaces']

for folder in mssparkutils.fs.ls("Files"):
    if folder.name in folders_to_move:
        print(folder.name + ' will be moved')
        src_path = folder.path
        dest_path = src_path.replace('Files/' + folder.name , 'Files/history/' + folder.name + '/old'   )
        print(dest_path)
        mssparkutils.fs.mv(src_path, dest_path, True)

StatementMeta(, 29e2abb3-de29-4007-8b21-301face677c5, 4, Finished, Available, Finished)